In [11]:
import pandas as pd

In [16]:
# Load the three datasets
average_hours = pd.read_csv('Resources/average-usual-weekly-hours-worked-women-15-years-and-older.csv')
employment_ratio = pd.read_csv("Resources/female-employment-to-population-ratio.csv")
wage_gap = pd.read_csv("Resources/gender-wage-gap-oecd.csv")

# Check the data types of the columns
print("average_hours columns:", average_hours.dtypes)
print("employment_ratio columns:", employment_ratio.dtypes)
print("wage_gap columns:", wage_gap.dtypes)

# Merge the three DataFrames into one\n",
merged_df = pd.merge(average_hours, employment_ratio, on=['Entity', 'Code', 'Year'], how='outer')
merged_df = pd.merge(merged_df, wage_gap, on=['Entity', 'Code', 'Year'], how='outer')

# Sort the combined dataframe by Entity and Year\n",
merged_df = merged_df.sort_values(by=['Entity', 'Year'])

average_hours columns: Entity                                                                            object
Code                                                                              object
Year                                                                               int64
Average weekly hours worked (women, 15+) (OECD Labor Force Statistics (2017))    float64
dtype: object
employment_ratio columns: Entity                                                                  object
Code                                                                    object
Year                                                                     int64
Employment to population ratio, 15+, female (%) (national estimate)    float64
dtype: object
wage_gap columns: Entity                          object
Code                            object
Year                             int64
Gender wage gap (OECD 2017)    float64
dtype: object
